### 5. 

### 1. 데이터 읽어오기

In [ ]:
import pandas as pd

raw_data_train = pd.read_csv('train.csv')
raw_data_test = pd.read_csv('test.csv')
raw_data_submission = pd.read_csv('sample_submission.csv')

### 2. 데이터 확인

### 3. 데이터 전처리

In [ ]:
train = raw_data_train.copy()
test = raw_data_test.copy()
submission = raw_data_submission.copy()

columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train = train[columns + ['Survived']]
test = test[columns]

In [ ]:
# Null 처리
mean_age = train['Age'].mean()
mean_fare = train['Fare'].mean()

train.loc[:, 'Age'] = train['Age'].fillna(mean_age)
test.loc[:, 'Age'] = test['Age'].fillna(mean_age)
train.loc[:, 'Fare'] = train['Fare'].fillna(mean_fare)
test.loc[:, 'Fare'] = test['Fare'].fillna(mean_fare)

# 이상치 제거
train = train[train['Parch'] <= 5]       
train = train[train['Fare'] <= 300] 

In [ ]:
train['Sex'] = train['Sex'].apply(lambda x: 0 if x == 'female' else 1)
test['Sex'] = test['Sex'].apply(lambda x: 0 if x == 'female' else 1)

train = pd.get_dummies(train, columns=['Embarked'], drop_first=True)
test = pd.get_dummies(test, columns=['Embarked'], drop_first=True)

In [ ]:
train_x = train.drop('Survived', axis = 1)
train_y = train['Survived']

from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y  = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(train_x,list(train_y))

X_resampled['Survived'] = y_resampled
train_dataset = X_resampled

### 4. 데이터 학습

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=6, random_state=0)
model.fit(train_dataset.drop(columns='Survived'),train_dataset['Survived'])

In [ ]:
y_pred = model.predict(val_x)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(val_y, y_pred))

from sklearn.metrics import classification_report
print(classification_report(val_y, y_pred))

### 5. CSV파일로 저장

In [ ]:
y_pred = model.predict(test)  
submission['Survived'] = y_pred
submission['Survived'] = submission['Survived'].apply(lambda x: 1 if x >= 0.5 else 0)
submission.head(15)

In [ ]:
submission.to_csv('submission.csv', index=False)